## Automate Event Planning

In [1]:
#!pip install -U crewai crewai_tools langchain_community

In [2]:
from crewai import Agent, Task, Crew
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

# from langchain_core.messages import HumanMessage
# message = HumanMessage(
#     content="Translate this sentence from English to French. I love programming."
# )
# r = default_llm.invoke([message])
# print(r)

## Tools

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Agents

In [4]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm = default_llm,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm = default_llm,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm = default_llm,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Venue Pydantic Object

In [5]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Tasks

In [6]:
output_json_file="venue_details.json"
output_md_file="marketing_report.md"

venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file=output_json_file,  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True, # the execution will ask for your input before it finishes running.
    async_execution=True, # the task can run in parallel with the tasks which come after it
    agent=logistics_manager
)

marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file=output_md_file,  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Crew

*Note:* Since you set async_execution=True for logistics_task and marketing_task tasks, now the order for them does not matter in the tasks list.

In [7]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

In [8]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
To find a suitable venue for the Tech Innovation Conference in San Francisco, I should start by identifying potential venues that meet the event's theme, size, and budget constraints. This will involve searching for venues that are known for hosting tech conferences and are located in San Francisco.

Action: Search the internet
Action Input: {"search_query": "best venues for tech conferences in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: Top 12 tech conferences in San Francisco for 2024 ; Developer Week. 21st - 23rd February 2024. Oakland Convention Center ; Women Impact Tech. July ...
---
Title: The 25 Best Event Venues in San Francisco - Bizzab

In [9]:
import json
from pprint import pprint

with open(output_json_file) as f:
   data = json.load(f)

pprint(data)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

To effectively market the Tech Innovation Conference and engage potential attendees, the first step is to gather information about the venues to highlight them in our marketing materials. We need to ensure that we have the most up-to-date and detailed information about each venue.

Action: Read website content
Action Input: {"website_url": "http://www.cityviewmetreon.com"}
To effectively coordinate catering and equipment for the Tech Innovation Conference with 500 participants on 2024-09-15, I need to first finalize the venue. The venue should comfortably accommodate 500 participants and offer necessary amenities such as catering and equipment setup.

I will start by selecting the most suitable venue from the provided list. Given the requirements, I will focus on venues with capacities that comfortably exceed 500 participants to ensure ample space and avoid overcrowding.

Thought: I need to gather detailed information about available venues to make an informed decision.

Action: Read w

In [11]:
from IPython.display import Markdown
Markdown(output_md_file)

## Marketing Activities and Attendee Engagement Report for Tech Innovation Conference

### Venue Information

#### City View at METREON
- **Location**: SoMa
- **Capacity**: 2,000
- **More Info**: [City View at METREON](http://www.cityviewmetreon.com)
- **Description**: Centrally located and recently renovated, City View at METREON offers over 31,000 square feet of divisible space. It features floor-to-ceiling windows with unobstructed views of San Francisco's skyline and an expansive outdoor terrace, making it a versatile venue for various events.

#### Gallery 308 at Fort Mason
- **Location**: Marina District
- **Capacity**: 430
- **More Info**: [Gallery 308 at Fort Mason](http://www.fortmason.org/venue/gallery-308)
- **Description**: Originally used for maritime trade and repair, Gallery 308 features panoramic views of San Francisco Bay and the Golden Gate Bridge. The space has an open floor plan, state-of-the-art lighting, polished concrete floors, and multi-purpose blackout shades, ideal for art exhibits, receptions, and conferences.

### Marketing Strategy

1. **Social Media Campaigns**:
   - **Platforms**: Instagram, Facebook, Twitter
   - **Content**: Highlighting venue features, speaker profiles, event agenda, interactive posts to engage potential attendees.
   - **Frequency**: Daily posts leading up to the event, with increased frequency in the final week.

2. **Email Marketing**:
   - **Audience**: Previous attendees, industry professionals, tech enthusiasts.
   - **Content**: Event details, early bird registration offers, exclusive content previews.
   - **Frequency**: Weekly newsletters, with reminders and updates as the event date approaches.

3. **Partnerships and Sponsorships**:
   - **Tech Companies**: Partner with leading tech companies for sponsorships and cross-promotion.
   - **Media Partners**: Collaborate with tech blogs, online magazines, and local news outlets for coverage.

4. **Content Marketing**:
   - **Blog Posts**: Articles on the latest tech trends, interviews with keynote speakers, behind-the-scenes content.
   - **Video Content**: Teasers of the venue, speaker introductions, testimonials from past events.

5. **SEO and SEM**:
   - **Keyword Optimization**: Optimize the event website and blog content for relevant keywords to improve organic search rankings.
   - **Paid Advertising**: Google Ads campaigns targeting keywords related to tech conferences, innovation events, and the San Francisco area.

### Attendee Engagement

1. **Interactive Sessions**:
   - **Q&A Panels**: Allow attendees to engage with speakers and ask questions in real-time.
   - **Workshops**: Hands-on sessions where attendees can learn new skills and technologies.

2. **Networking Opportunities**:
   - **Networking Lounges**: Dedicated spaces for attendees to connect and collaborate.
   - **App Integration**: Use event apps to facilitate networking by matching attendees with similar interests.

3. **Feedback Mechanisms**:
   - **Surveys**: Pre- and post-event surveys to gather attendee feedback and improve future events.
   - **Live Polls**: Real-time polls during sessions to keep the audience engaged and gather instant feedback.

4. **Exclusive Access**:
   - **VIP Packages**: Offer VIP tickets with benefits such as exclusive meet-and-greets, access to VIP lounges, and premium seating.
   - **Early Access**: Provide early access to the venue and sessions for VIP ticket holders.

By implementing this comprehensive marketing strategy and focusing on attendee engagement, we aim to maximize event exposure and participation, ultimately achieving our goal of engaging at least 500 potential attendees.

In [12]:
print(event_management_crew.usage_metrics)

{'total_tokens': 20338, 'prompt_tokens': 18131, 'completion_tokens': 2207, 'successful_requests': 4}


This is the agent final answer: **City View at METREON - Event Logistics Information**

**Availability of Catering Services:**
- It is highly recommended that clients select a caterer from City View’s Preferred Vendors list. This list comprises caterers who have an established relationship with City View, are familiar with the facilities and required rules and regulations, and have excellent reputations.
- There is a catering kitchen available on-site, located adjacent to the main venue, featuring space for food preparation, storage, and staging. The kitchen is a prep and production space with connections for electric and gas equipment and a full walk-in refrigerator. The area is recommended to be used primarily for finishing, plating, and bussing.
- If alcohol is to be served at your event, an alcohol catering permit must be completed, approved, and submitted to Management at least 15 days prior to your event.

**Equipment Setup and Additional Services:**
- Load-In/Load-Out: There is 